In [ ]:
'''This is a collection of code I have written while working with the Battle.net API. 
Before and after each script is a comment explaining basic functionality, and a comment explaining the code in more depth, respectively.

(NOTE: NONE OF THESE SCRIPTS ARE MEANT TO BE RUN IN THIS NOTEBOOK)'''

In [ ]:
#Ran this loop to keep my access token up to date and store it in a text file

In [ ]:
import requests as rq
import time
import asyncio


async def main(): 
    while True:

        token_data = token_data = rq.post(r'https://oauth.battle.net/token',
                        auth=('user','pass'),
                        data={'grant_type':'client_credentials'}).json()

        print(token_data)

        if token_data:

            token = token_data['access_token']


            with open('token.txt','w') as token_file:
                token_file.write(token)
                print('written')
            print('big sleep')
            time.sleep(token_data['expires_in']-100)
        
        time.sleep(5)


asyncio.run(main())

In [ ]:
'''in Bnets API, basic authentication is used to retrieve an access token, that token then allows the use of other API features(bearer auth).
My script is a loop that: 
    grabs a token granted response from bnets API, 
    writes to a text file with the access token, 
    waits for an amount of time that is derived from the response's expiration time on the token,
    and repeats.'''

In [ ]:
#This script was used in multiple variations to navigate and understand the structure of the data coming from the API

In [ ]:
import requests as rq

with open('token.txt','r') as file:
    token = file.read()


data = rq.get(r'https://us.api.blizzard.com/hearthstone/cardbacks', headers={'Authorization': f'Bearer {token}'}).json()

for i in data['cardBacks']:
    print( i['name']['en_US'])

In [ ]:
'''This version of the script just grabs the name of each card back on the first page of the cardbacks database
as for a multi-page version of this logic, I was able to use a variation of this script to see that there is a 'pagecount' in the data.
This will allow me to just conveniently grab that number and iterate through that pages.'''

In [ ]:
#this script will grab specific data from each page

In [ ]:
import pandas as pd
import requests as rq
import asyncio

with open('token.txt', 'r') as file:
    token = file.read()


def fetch_page(page, predf):
    data = rq.get(
        'https://us.api.blizzard.com/hearthstone/cardbacks',
        headers={'Authorization': f'Bearer {token}'},
        params={'page': page}
    ).json()

    for i in data.get('cardBacks', []):
        predf.append({
            'Name': i['name']['en_US'],
            'Card Back': i['image'],
            'Text': i['text']['en_US']
        })


async def main():
    predf = []
    async with asyncio.TaskGroup() as tg:
        for page in range(1, 8):
            tg.create_task(asyncio.to_thread(fetch_page, page, predf))
    df = pd.DataFrame(predf)
    return df


print(asyncio.run(main()))

In [ ]:
'''I used asyncios TaskGroup context manager to run the fetching of each page in a different thread at the same time, this is for speed and efficiency.
This thread is a step in the development process where the end goal is having an excel sheet where I have some text data, and displayed images.
The 'image' part of the data right now is just Amazon Cloudflare links.

This script was useful for making sure everything was in order before saving it to an excel file'''

In [ ]:
#This script replaced image links in my excel file with embedded images. Asynchronous input/output was used for speed and efficiency.

In [ ]:
import openpyxl.drawing
import openpyxl.drawing.image
import openpyxl.workbook
import asyncio, pandas as pd, aiohttp, PIL, openpyxl
from PIL import Image
from io import BytesIO
from openpyxl import workbook


book = openpyxl.load_workbook('images_replaced.xlsx')
sheet = book.active


async def fetch(session, url, idx):
    async with session.get(url) as response:
        content = await response.read()
        image = Image.open(BytesIO(content))
        sheet[f'C{idx+2}'] = ''
        openpyxl_image = openpyxl.drawing.image.Image(image)
        sheet.add_image(openpyxl_image, f'C{idx+2}')
        print(f'added {url} to {idx+2}')
        

async def main():
    df = pd.read_excel('Card Backs.xlsx')
    urls = urls = df['Card Back'].tolist()
    async with aiohttp.ClientSession() as session:
        tasks = [fetch(session, url, idx) for idx, url in enumerate(urls)]

        await asyncio.gather(*tasks)
    book.save('images_replaced_with_images.xlsx')
    

asyncio.run(main())

In [ ]:
'''I used pandas to read the excel file and create a list, in order, of the urls from my images column. 

I then used asyncio to iterate over the urls, creating a task to get the image from the url, and replace the text in the cell with the image itself.
The loop that creates the tasks enumerates the urls to ensure the images are put in the correct cells.

I designed this to all happen asynchronously, and entirely in memory(including the images), for the sake of speed and efficiency.'''